# Sentiment Analysis

## Import statements

In [1]:
import pandas as pd
import re
from functools import reduce
from textblob import TextBlob

/Users/laureneterno/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


## Save excel files from data cleaning stage into DataFrames and name the timestamp column

In [2]:
nike = pd.read_excel('nike.xlsx')
starbucks = pd.read_excel('starbucks.xlsx')
target = pd.read_excel('target.xlsx')

nike = nike.rename(columns = {'Unnamed: 0': 'timestamp'})
starbucks = starbucks.rename(columns = {'Unnamed: 0': 'timestamp'})
target = target.rename(columns = {'Unnamed: 0': 'timestamp'})

## Define getPolarity function that uses the TextBlob library to perform sentiment analysis on the comments

In [3]:
def getPolarity(text):
   return TextBlob(text).sentiment.polarity

## Define sentiment analysis function that cleans the string data and assigns each date a polarity score using the TextBlob library

In [4]:
def sentiment_analysis(df):
    list = []
    for x in df['comments']:
    
        # changes all of the characters to lowercase
        x = str.lower(x)
    
        # removes all special characters
        x = re.sub(r'[^a-zA-Z0-9\s]+', '', x)
    
        # adds the modified column values to a list
        list.append(x)

        # adds the nike_list to a new dataframe
        new_df = pd.DataFrame(list, columns =['comments'])

    # adds the timestamps to the new dataframe
    new_df = new_df.join(df['timestamp'])

    # performs sentiment analysis on the comments from each day
    new_df['polarity_score'] = new_df['comments'].apply(getPolarity)

    # rearrange column order
    new_df = new_df[['timestamp', 'polarity_score','comments']]

    # set timestamp as index
    new_df.set_index('timestamp')
    
    return new_df

## Creates the new dataframe using the sentiment analysis function on each company's dataframe

In [5]:
nike = sentiment_analysis(nike)

In [6]:
starbucks = sentiment_analysis(starbucks)

In [7]:
target = sentiment_analysis(target)

# Machine Learning

## Import Statements

In [8]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.metrics import mean_absolute_error

2023-12-06 20:35:22.228970: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


## save the stock data into csv files

In [9]:
nike_stock = pd.read_csv('nike_stock.csv')
starbucks_stock = pd.read_csv('starbucks_stock.csv')
target_stock = pd.read_csv('target_stock.csv')

## combine the stock data and sentiment analysis data together. we chose the close price to be our output variable for the LSTM models

In [10]:
def combine_and_clean(stock, sa):
    
    # drop columns not needed from initial data retreival
    stock = stock.drop(['Open', 'High', 'Low', 'Adj Close', 'Volume'], axis = 1)
    
    # change columns to datetime type
    sa['timestamp'] = pd.to_datetime(sa['timestamp'])
    stock['Date'] = pd.to_datetime(stock['Date'])
    
    # perform an inner join to combine the dataframes by the dates available for both the sentiment analysis and the stock data
    sa = sa.merge(stock, how = 'inner', left_on='timestamp', right_on = 'Date')
    
    # drop the date column as it is the same as the timestamp column (since the join was performed) and drop the comments column as the sentiment analysis was already completed
    sa = sa.drop(['Date', 'comments'], axis = 1)
    
    # rename the Close column to close to match the lowercase in the other column names
    sa = sa.rename(columns = {'Close': 'close'})
    
    # rename the timestamp column to date
    sa = sa.rename(columns = {'timestamp':'date'})
    
    # Convert 'date' to numerical feature (number of days since the start)
    sa['days_since_start'] = (sa['date'] - sa['date'].min()).dt.days
        
    #return the cleaned dataframe
    return sa

In [11]:
nike = combine_and_clean(nike_stock, nike)
starbucks = combine_and_clean(starbucks_stock, starbucks)
target = combine_and_clean(target_stock, target)

## Create a long short term memory model for each of the companies that uses the date (normalized as date since the earliest date) and polarity_score from sentiment analysis as input variables and the close price as the output variable

In [12]:
def model(df):
    # feature selection
    features = df[['days_since_start', 'polarity_score', 'close']].values
    
    # scale the values between 0 and 1 (this is optimal for LSTM)
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled = scaler.fit_transform(features)

    # Define nested function to create dataset with input features and target variable
    def create_dataset(data, length):
        # x represents the input features
        # y represents the output variables
        x, y = [], []
        for i in range(len(data) - length):
            a = data[i:(i + length), :]
            x.append(a)
            y.append(data[i + length, 2])  # 'close' is the third column
        return np.array(x), np.array(y)

    # determines the sequence length for each input
    length = 10

    # create dataset
    x, y = create_dataset(scaled, length)

    # split data into training and testing
    # test size is 20% of data
    # random_state = 42 means that the same training and testing sets are used across executions
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

    # build LSTM model
    # 2 LSTM layers 
    model = Sequential()
    model.add(LSTM(units=50, return_sequences=True, input_shape=(x_train.shape[1], x_train.shape[2])))
    model.add(LSTM(units=50))
    # dense output layer
    model.add(Dense(units=1))

    # compile
    model.compile(optimizer='adam', loss='mean_squared_error')

    # train
    model.fit(x_train, y_train, epochs=50, batch_size=32)

    # evaluate
    test_loss = model.evaluate(x_test, y_test)

    # define variable for test_loss output
    test_loss_output = f'Test Loss: {test_loss}'

    # predict
    predictions = model.predict(x_test)

    # unscale the data so that it is back in the original form
    predictions = scaler.inverse_transform(np.concatenate((x_test[:, -1, 0:2], predictions.reshape(-1, 1)), axis=1))[:, 2]
    
    # evaluate
    actual_close = scaler.inverse_transform(np.concatenate((x_test[:, -1, 0:2], y_test.reshape(-1, 1)), axis=1))[:, 2]
  
    mae = mean_absolute_error(actual_close, predictions)
    mae_output = f'Mean Absolute Error: {mae}'
    return predictions, test_loss_output, mae_output

In [13]:
model(nike)

Epoch 1/50
3/3 [==============================] - 3s 9ms/step - loss: 0.2173
Epoch 2/50
3/3 [==============================] - 0s 9ms/step - loss: 0.0918
Epoch 3/50
3/3 [==============================] - 0s 10ms/step - loss: 0.0484
Epoch 4/50
3/3 [==============================] - 0s 13ms/step - loss: 0.0519
Epoch 5/50
3/3 [==============================] - 0s 10ms/step - loss: 0.0501
Epoch 6/50
3/3 [==============================] - 0s 14ms/step - loss: 0.0360
Epoch 7/50
3/3 [==============================] - 0s 10ms/step - loss: 0.0313
Epoch 8/50
3/3 [==============================] - 0s 9ms/step - loss: 0.0313
Epoch 9/50
3/3 [==============================] - 0s 13ms/step - loss: 0.0307
Epoch 10/50
3/3 [==============================] - 0s 8ms/step - loss: 0.0277
Epoch 11/50
3/3 [==============================] - 0s 8ms/step - loss: 0.0261
Epoch 12/50
3/3 [==============================] - 0s 7ms/step - loss: 0.0273
Epoch 13/50
3/3 [==============================] - 0s 8ms/step - lo

(array([108.01542726,  98.5412247 , 102.20982773, 100.99398736,
         98.82535949, 110.3391464 ,  98.27381104, 100.08089034,
         97.64291405, 110.71809557, 114.37922221, 109.67295321,
        106.36720028, 110.38503832, 113.13555937, 109.40926233,
        106.92204085, 109.15304677]),
 'Test Loss: 0.019239556044340134',
 'Mean Absolute Error: 3.6612329058305906')

In [14]:
model(starbucks)

Epoch 1/50
3/3 [==============================] - 3s 7ms/step - loss: 0.3050
Epoch 2/50
3/3 [==============================] - 0s 11ms/step - loss: 0.1631
Epoch 3/50
3/3 [==============================] - 0s 10ms/step - loss: 0.0782
Epoch 4/50
3/3 [==============================] - 0s 10ms/step - loss: 0.0610
Epoch 5/50
3/3 [==============================] - 0s 10ms/step - loss: 0.0737
Epoch 6/50
3/3 [==============================] - 0s 9ms/step - loss: 0.0596
Epoch 7/50
3/3 [==============================] - 0s 8ms/step - loss: 0.0474
Epoch 8/50
3/3 [==============================] - 0s 10ms/step - loss: 0.0476
Epoch 9/50
3/3 [==============================] - 0s 10ms/step - loss: 0.0487
Epoch 10/50
3/3 [==============================] - 0s 9ms/step - loss: 0.0466
Epoch 11/50
3/3 [==============================] - 0s 7ms/step - loss: 0.0428
Epoch 12/50
3/3 [==============================] - 0s 7ms/step - loss: 0.0422
Epoch 13/50
3/3 [==============================] - 0s 7ms/step - lo

(array([100.23816096,  95.4026091 ,  94.34531086, 101.87500116,
        100.05231546,  99.58781405, 102.49934253,  98.43743494,
        103.43291968,  96.02916689, 105.89282643, 101.07954077,
         95.03499691, 100.2683296 , 102.1882648 ,  99.96601979,
         99.60846301, 100.51413095, 102.2259052 , 101.94899345,
         96.41532084, 101.34241817]),
 'Test Loss: 0.018969757482409477',
 'Mean Absolute Error: 2.6421495361778913')

In [15]:
model(target)

Epoch 1/50
4/4 [==============================] - 3s 8ms/step - loss: 0.1341
Epoch 2/50
4/4 [==============================] - 0s 9ms/step - loss: 0.0641
Epoch 3/50
4/4 [==============================] - 0s 8ms/step - loss: 0.0553
Epoch 4/50
4/4 [==============================] - 0s 7ms/step - loss: 0.0356
Epoch 5/50
4/4 [==============================] - 0s 7ms/step - loss: 0.0317
Epoch 6/50
4/4 [==============================] - 0s 7ms/step - loss: 0.0275
Epoch 7/50
4/4 [==============================] - 0s 7ms/step - loss: 0.0273
Epoch 8/50
4/4 [==============================] - 0s 8ms/step - loss: 0.0263
Epoch 9/50
4/4 [==============================] - 0s 8ms/step - loss: 0.0258
Epoch 10/50
4/4 [==============================] - 0s 7ms/step - loss: 0.0252
Epoch 11/50
4/4 [==============================] - 0s 9ms/step - loss: 0.0254
Epoch 12/50
4/4 [==============================] - 0s 8ms/step - loss: 0.0252
Epoch 13/50
4/4 [==============================] - 0s 8ms/step - loss: 0.

(array([132.72089959, 148.47239925, 147.35265885, 121.31633729,
        127.37405059, 135.65514223, 114.92178903, 133.10141759,
        155.36987382, 156.62607164, 125.84379527, 129.47000323,
        121.00571473, 125.52150557, 157.13084725, 128.48497834,
        160.02976199, 109.16155503, 145.38470538, 153.02582692,
        160.73220885, 119.40234434, 118.9564242 , 120.65275154,
        113.38125301, 132.81725659, 125.68519133]),
 'Test Loss: 0.014349307864904404',
 'Mean Absolute Error: 6.700150104850051')